In [ ]:
import logging
import os
import sys
import json
from typing import List

from langchain.agents.agent import AgentExecutor
from langchain_core.tools import Tool, StructuredTool
from langchain_aws.agents.bedrock.agent_base import agent_tool, BedrockAgentMetadata, BedrockAgentBase
from langchain_aws.agents.bedrock.agent_executor import BedrockAgentExecutor
from langchain_aws.agents.bedrock.bedrock_agent import BedrockAgent

logging.basicConfig(format='%(message)s', level=logging.INFO)

In [ ]:

agent_name: str = "MortgageEvaluatorAgent_1"
agent_instruction: str = "You are an agent who helps with getting the mortgage rate based on the current asset valuation"
agent_region: str = "us-east-1"

@agent_tool(action_group='AssetDetail', action='getAssetValue')
def getAssetValue(
        asset_holder_id: str = ' '
) -> str:
    """
    Get the asset value for an owner id

    Args:
        asset_holder_id: id of the owner holding the asset
    Returns:
        str -> the valuation of the asset
    """
    return f"The total asset value for {asset_holder_id} is 100K"


@agent_tool
def getMortgageRate(
        asset_holder_id: str = ' ',
        asset_value: str = ' '
) -> str:
    """
    Get the mortage rate based on asset value

    Args:
        asset_holder_id: id of the owner holding the asset
        asset_value: asset value which is used to get the mortgage rate
    Returns:
        str -> the calculated mortage rate based on the asset value
    """
    return f"The mortgage rate for {asset_holder_id} with asset value of {asset_value} is 8.87%"

agent_tools: List[Tool] = [getAssetValue, getMortgageRate]

def handle_trace(event=dict):
    logging.info("\n********************** Event **********************\n")
    logging.info(f"{json.dumps(event, indent=4)} \n")

In [ ]:
bedrock_agent = BedrockAgent(
    agent_name=agent_name,
    agent_instruction=agent_instruction,
    agent_region=agent_region,
    agent_tools=agent_tools,
    trace_handler=handle_trace
)

agent_executor = BedrockAgentExecutor.from_agent_and_tools(
    agent=bedrock_agent,
    verbose=True,
    tools=agent_tools,
    return_intermediate_steps=True,
    max_iterations=8
)

In [ ]:
agent_executor.invoke(agent_input="what is my mortgage rate for id AVC-1234")

In [ ]:
bedrock_agent.delete()